In [1]:
# 엑셀 파일을 불러오기 위한 xlrd
# !pip install xlrd
import xlrd

workbook = xlrd.open_workbook("train_test.xlsx")

# train
worksheet = workbook.sheet_by_index(0) # 엑셀 파일의 train sheet
train = worksheet.col_values(3) 
y_train = worksheet.col_values(2)

# test
worksheet = workbook.sheet_by_index(1) # 엑셀 파일의 test sheet
test = worksheet.col_values(3) 
y_test = worksheet.col_values(2)

In [2]:
# 정수 인코딩
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(train)

# 벡터화
x_train = tokenizer.texts_to_sequences(train)
x_test = tokenizer.texts_to_sequences(test)

In [3]:
# 총 단어 수
total_word = len(tokenizer.word_index)

# 단어 집합의 크기
vocab_size = total_word+1

import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

In [4]:
import pickle

with open('tokenizer.p', 'wb') as file: # 파일을 바이너리 쓰기 모드(wb)로 열기 
    pickle.dump(tokenizer, file) # 저장하기

In [5]:
import pandas as pd

# 데이터의 최대 길이 확인
# train + test 데이터 합치기
all_data = train + test

max_len = 0
for i in all_data:
    if max_len <= len(i):
        max_len = len(i)
        
print(max_len)

264


In [6]:
# 패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen = 100)
x_test = pad_sequences(x_test, maxlen = 80)

In [7]:
x_test.shape

(49572, 80)

In [8]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape, max_len, vocab_size

((148738, 100), (49572, 80), (148738,), (49572,), 264, 39682)

In [9]:
# 모델 구축
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.models import Sequential

In [10]:
model__ = Sequential()

model__.add(Embedding(vocab_size, 128)) # 임베딩
model__.add(LSTM(64, activation='relu'))
model__.add(Dropout(0.5))
model__.add(Dense(32, activation='relu'))
model__.add(Dense(1, activation='sigmoid')) 

model__.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
model__.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         5079296   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 5,130,817
Trainable params: 5,130,817
Non-trainable params: 0
_________________________________________________________________


In [16]:
# 모델 학습
# loss 값이 갑자기 커지는 부분이 있어 에포크를 30, 100개 마다 확인하게 설정, 과적합 발생 가능성.
model__.fit(x_train, y_train, epochs = 30, batch_size = 100, validation_split = 0.2)

Epoch 1/30
1190/1190 [==============================] - 184s 155ms/step - loss: 2057.1316 - accuracy: 0.8452 - val_loss: 0.3422 - val_accuracy: 0.8506
Epoch 2/30
1190/1190 [==============================] - 183s 153ms/step - loss: 135.6443 - accuracy: 0.8592 - val_loss: 0.4437 - val_accuracy: 0.8549
Epoch 3/30
1190/1190 [==============================] - 180s 151ms/step - loss: 13263.4209 - accuracy: 0.8726 - val_loss: 4.8256 - val_accuracy: 0.8445
Epoch 4/30
1190/1190 [==============================] - 179s 151ms/step - loss: 176649.1875 - accuracy: 0.8777 - val_loss: 311.7954 - val_accuracy: 0.8619
Epoch 5/30
1190/1190 [==============================] - 180s 151ms/step - loss: 5.3410 - accuracy: 0.8831 - val_loss: 0.3405 - val_accuracy: 0.8526
Epoch 6/30
1190/1190 [==============================] - 181s 153ms/step - loss: 1157.0393 - accuracy: 0.8877 - val_loss: 65.8061 - val_accuracy: 0.8620
Epoch 7/30
1190/1190 [==============================] - 180s 151ms/step - loss: 3546905088.0

In [17]:
model__.save("model.h5")

In [18]:
import xlrd

workbook = xlrd.open_workbook("test.xlsx")

# test
worksheet = workbook.sheet_by_index(0) # test sheet에 접근합니다.
test = worksheet.col_values(0) # 기본 data
test_morpheme = worksheet.col_values(1) # kkma를 이용하여 형태소 분석한 결과
test_syllable = worksheet.col_values(2) # 음절별로 나눈 결과

In [19]:
# tokenzier load

with open('tokenizer.p', 'rb') as file: # 파일을 바이너리 읽기 모드(rb)로 열기
    tokenizer = pickle.load(file)

In [20]:
test = list(map(str, test))

# 벡터화
test_data = tokenizer.texts_to_sequences(test)

In [21]:
max_len = 0
for i in test_data:
    if max_len <= len(i):
        max_len = len(i)
        
# 패딩
from tensorflow.keras.preprocessing.sequence import pad_sequences

test_data = pad_sequences(test_data, maxlen = max_len)

In [22]:
y_pred = model__.predict_classes(test_data).reshape(-1)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [23]:
print(y_pred)

[0 0 0 ... 1 0 0]


In [24]:
import openpyxl

In [26]:
from openpyxl import Workbook

write = Workbook()
write_ = write.create_sheet("y값")

write_ = write.active

# 값 추가 / 행 단위
for i in y_pred:
    write_.append([i])
    
# 저장
write.save("20165153 이재성 예측한 값.xlsx")